In [16]:
import torch
import pandas as pd
import numpy as np

In [23]:
import os

In [27]:
os.getcwd()

'E:\\PyTorch Sessions'

In [25]:
os.chdir('E:/PyTorch Sessions')

# Some Basic Operations

In [3]:
x = torch.empty(5,3)

In [4]:
print(x)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [5]:
y = torch.ones(5,3)

In [6]:
print(y)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [9]:
z = torch.rand(5,3)

In [10]:
y + z

tensor([[1.2357, 1.5211, 1.2913],
        [1.5734, 1.7517, 1.2776],
        [1.0321, 1.0705, 1.2155],
        [1.0373, 1.5092, 1.8708],
        [1.9923, 1.7955, 1.8832]])

In [11]:
type(z)

torch.Tensor

In [12]:
torch.add(y,z)

tensor([[1.2357, 1.5211, 1.2913],
        [1.5734, 1.7517, 1.2776],
        [1.0321, 1.0705, 1.2155],
        [1.0373, 1.5092, 1.8708],
        [1.9923, 1.7955, 1.8832]])

In [14]:
y.add_(z)

tensor([[1.2357, 1.5211, 1.2913],
        [1.5734, 1.7517, 1.2776],
        [1.0321, 1.0705, 1.2155],
        [1.0373, 1.5092, 1.8708],
        [1.9923, 1.7955, 1.8832]])

In [15]:
y.numpy()

array([[1.2357426, 1.5211222, 1.2912881],
       [1.5733962, 1.7517278, 1.2776225],
       [1.032126 , 1.0705348, 1.2154775],
       [1.0373068, 1.5091655, 1.8707666],
       [1.992306 , 1.795492 , 1.8831587]], dtype=float32)

# Autograd - Automatic Differentiation

In [391]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [392]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [393]:
print(y.grad_fn)

In [395]:
y.requires_grad

True

In [394]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [405]:
z.requires_grad

True

In [398]:
print(x.grad)

None


In [399]:
print(y.grad)

None


In [400]:
out.backward()

In [403]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


# A MultiClass Classifier with PyTorch on Iris Dataset

### Read Data

In [135]:
data = pd.read_csv('E:/PyTorch Sessions/iris.data', header = None)

In [116]:
data.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [61]:
len(data)

150

In [137]:
data[4] = data[4].map({'Iris-setosa' : 0, 'Iris-versicolor' : 1, 'Iris-virginica' : 2, })

In [138]:
data[4].value_counts()

2    50
1    50
0    50
Name: 4, dtype: int64

### Split the data into train, val, and test

In [79]:
Train = data.sample(frac=0.8,random_state=200)
test = data.drop(Train.index)

In [80]:
len(train)

120

In [81]:
len(test)

30

In [82]:
train = Train.sample(frac=0.9,random_state=200)

val = Train.drop(train.index)


In [83]:
len(train)

108

In [84]:
len(val)

12

In [139]:
train[4].value_counts()

0    38
1    36
2    34
Name: 4, dtype: int64

### Save them into separate files (It will help later)

In [327]:
train.to_csv('train.csv', index = False, header = False)
test.to_csv('test.csv', index = False, header = False)
val.to_csv('val.csv', index = False, header = False)

In [328]:
from torch.utils.data import Dataset

from torch.utils.data import DataLoader

from torchvision import transforms

### Create your dataset class

In [382]:
class IrisDataset(Dataset):
    
    def __init__(self, file_path, transform = None):
        self.data_frame = pd.read_csv(file_path, header = None)
        self.transform = transform
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self,idx):
        X = self.data_frame.iloc[idx, :4].values
        y = self.data_frame.iloc[idx,4]
        X = (X - X.mean())/X.std()
        
        return X, y
    
    

### Create dataset objects

In [383]:
train_dataset = IrisDataset('train.csv')
val_dataset = IrisDataset('val.csv')
test_dataset = IrisDataset('test.csv')

### Set Parameters

In [384]:
learning_rate = 0.01
batch_size = 20
epochs = 20

### Instantiate DataLoader objects

In [385]:
training_generator = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

val_generator = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

testing_generator = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

In [386]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

### Design your Neural Network

In [387]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3) 
        self.fc3 = nn.Linear(3, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=3, bias=True)
)


### Define loss criteria and optimizer

In [388]:
optimizer = optim.Adam(net.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

### Implement your training loop

In [390]:
net = Net()
net.train()
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(training_generator):
        inputs, labels = data
      #  print(labels)
        inputs, labels = (inputs).type(torch.FloatTensor), (labels)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(running_loss/batch_size)
    
    print('----------Epoch Complete---------')
    

0.34462822675704957
----------Epoch Complete---------
0.3452052891254425
----------Epoch Complete---------
0.3414564967155457
----------Epoch Complete---------
0.3440441846847534
----------Epoch Complete---------
0.34088918566703796
----------Epoch Complete---------
0.3466432452201843
----------Epoch Complete---------
0.34030720591545105
----------Epoch Complete---------
0.3466417729854584
----------Epoch Complete---------
0.34722251296043394
----------Epoch Complete---------
0.349811053276062
----------Epoch Complete---------
0.34203571677207945
----------Epoch Complete---------
0.34521105885505676
----------Epoch Complete---------
0.3371326714754105
----------Epoch Complete---------
0.34405208230018614
----------Epoch Complete---------
0.34664379954338076
----------Epoch Complete---------
0.3362739831209183
----------Epoch Complete---------
0.34405922293663027
----------Epoch Complete---------
0.34146377444267273
----------Epoch Complete---------
0.3382852464914322
----------Epoch Co

### Save your Model

In [309]:
torch.save(net, 'E:/PyTorch Sessions/Simple_Net.pt')

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Load Model and Predict

In [310]:
Loaded_Net = torch.load('E:/PyTorch Sessions/Simple_Net.pt')

In [319]:
correct = 0
total = 0
with torch.no_grad():
    for data in testing_generator:
        inputs, labels = data
        inputs = (inputs).type(torch.FloatTensor)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(predicted)

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
